In [1]:
import nbimporter
from Cleaning_Dataset import clean_diamonds, standard

Importing Jupyter notebook from Cleaning_Dataset.ipynb


In [9]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [3]:
original_train = pd.read_csv("../Data/train.csv")

In [4]:
original_test = pd.read_csv("../Data/test.csv")

In [5]:
original_train_cleaned = clean_diamonds(original_train)

In [6]:
original_test_cleaned = clean_diamonds(original_test)

In [7]:
X = original_train_cleaned.drop("price", axis = 1)
y = original_train_cleaned["price"]
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state=123)

In [11]:
model = GradientBoostingRegressor(n_estimators=3000)

params = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
          'max_depth': [4, 6],
          'min_samples_leaf': [3, 5, 9, 17],
          'max_features': [1, 0.3, 0.1]}
grid_search = GridSearchCV(model,
                           param_grid=params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 38.5min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 91.7min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 101.1min finished


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(n_estimators=3000),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.05, 0.02, 0.01],
                         'max_depth': [4, 6], 'max_features': [1, 0.3, 0.1],
                         'min_samples_leaf': [3, 5, 9, 17]},
             verbose=1)

In [12]:
grid_search.best_score_

0.9927948047735313

In [15]:
best_gb = grid_search.best_estimator_
best_gb.score(X_test, y_test)

0.9926922329392379

In [16]:
grid_search.best_params_

{'learning_rate': 0.02,
 'max_depth': 6,
 'max_features': 0.3,
 'min_samples_leaf': 5}

### Real Test

In [24]:
prediction = best_gb.predict(original_test_cleaned)

In [25]:
len(prediction)

13485

In [26]:
GB_df = pd.DataFrame(prediction, columns = ["price"])

In [27]:
GB_df["id"] = RF_df.index

In [28]:
GB_df = GB_df[["id", "price"]]

In [29]:
GB_df.to_csv(r"GB_df.csv", index = False)

In [30]:
!mv GB_df.csv ../Predictions/GB_df.csv